In [24]:
import numpy as np
import pandas as pd

In [25]:
mean = [0, 0, 0]
cov = [
    [1, 1, 0], 
    [1, 2, 1], 
    [0, 1, 2]
]
num_samples = 1000
abh = np.random.multivariate_normal(mean, cov, num_samples)
abh_data = pd.DataFrame(abh, columns=['x_1', 'x_2', 'x_3'])

In [26]:
abh_data['x_4'] = np.random.binomial(1, p=1/2, size=num_samples)
abh_data['y'] = 0
abh_data.head()

,x_1,x_2,x_3,x_4,y
0,-1.832650,-2.938742,0.238920,0,0
1,-0.483841,-0.946960,-0.552063,0,0
2,-1.126962,-0.488758,1.951238,0,0
3,0.063783,-0.155246,1.258962,0,0
4,-1.186559,0.652666,2.080017,1,0


In [27]:
mean = [6, 6, 6]
cov = [
    [1, 0, 0], 
    [0, 1, 0], 
    [0, 0, 1]
]
mar = np.random.multivariate_normal(mean, cov, num_samples)
mar_data = pd.DataFrame(mar, columns=['x_1', 'x_2', 'x_3'])

In [28]:
mar_data['x_4'] = np.random.binomial(1, p=1/3, size=num_samples)
mar_data['y'] = 1
mar_data.head()

,x_1,x_2,x_3,x_4,y
0,4.340111,6.270967,6.490997,0,1
1,6.204159,6.261467,6.133126,0,1
2,4.974548,5.562584,7.595021,1,1
3,4.965265,6.466328,4.277239,0,1
4,4.625105,4.235930,8.268446,0,1


In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

train_data = pd.concat([mar_data, abh_data], axis=0)
train_data = train_data.reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(train_data[['x_1', 'x_2', 'x_3', 'x_4']], train_data['y'], stratify=train_data['y'], train_size=0.8)


In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

categorical = ['x_4']
numeric = ['x_1', 'x_2', 'x_3']
column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), categorical),
    ('scaling', StandardScaler(), numeric),
    # ('other',  'passthrough', other)
])

X_train = column_transformer.fit_transform(X_train)
X_test = column_transformer.transform(X_test)

In [31]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
pred = logreg.predict(X_test)

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

1.0

In [33]:
test_data = pd.read_csv('tangerines.csv')
test_data

,x_1,x_2,x_3,x_4
0,1,-1,3,1
1,3,4,3,0
2,7,1,2,1
3,3,6,0,0
4,-2,1,0,0
5,4,2,0,1
6,5,4,3,1
7,4,2,3,0
8,-3,6,4,0
9,3,3,3,1


In [34]:
test_data = column_transformer.transform(test_data)
pred = logreg.predict(test_data) + 1
print(' '.join([str(kek) for kek in pred]))

1 1 2 1 1 1 2 1 1 1


И вот тут была ошибка. На тестовых данных использовал `fit_transform` вместо `transform`.